In [10]:

from DescritoresDeImagem.readwritendarray import extractimagesdescriptor, readfilendarray
from DescritoresDeImagem.utils import lpq_function

# Standart Scale, SVC and KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib import pyplot as plt
from scipy.stats import loguniform

import numpy as np
import pickle


In [ ]:
#GLCM

distances=[1,2,4,8,16,32,64,128,256]
angles=np.deg2rad([0,45,90,180,270])


#LBP
Winsize=25

decorr=1

CVPARTS=5 # number of CV parts
RUN=True

if RUN:
    function_glcm=extractimagesdescriptor("DadosExtraidos/Y_Resultado.txt","DadosExtraidos/X_GLCM.txt","BasedeDados/*.bmp","GLCM",)
    function_glcm(angles=angles,distances=distances)

    function_lbp=extractimagesdescriptor(file="DadosExtraidos/Y_Resultado.txt",filename="DadosExtraidos/X_LBP.txt",pathimg="BasedeDados/*.bmp",descriptor="LBP")
    function_lbp(winSize=Winsize,decorr=decorr)



X_lbp=readfilendarray(filename="DadosExtraidos/X_LBP.txt")

X_lpq=lpq_function("BasedeDados/*.bmp",P=30,R=6)

X_glcm=readfilendarray(filename="DadosExtraidos/X_GLCM.txt")

y=readfilendarray(filename="DadosExtraidos/Y_Resultado.txt",data="s")

y = y.reshape(-1, 1)

x_lpq_lpq_glcm=np.concatenate((X_lbp,X_lpq,X_glcm,y),axis=1)

X_train_model,X_test_model= train_test_split(x_lpq_lpq_glcm,test_size=0.2,train_size=0.8,random_state=999)



FUNCTION GLCM
FUNCTION LBP


In [13]:
y_train=X_train_model[:,-1]

y_test=X_test_model[:,-1]

X_train_model=X_train_model[:,:-1]

X_test_model=X_test_model[:,:-1]



In [14]:
def randomgridsearchotimizator(n_iter:int,scoring:str,cvparts,pipe,parameters):
    return RandomizedSearchCV(estimator=pipe,param_distributions=parameters,n_iter=n_iter,scoring=scoring,cv=cvparts,n_jobs=6,error_score='raise')

In [27]:
pipe=Pipeline(steps=[("scalar",StandardScaler()),
                     ("model",SVC())])


parameters = {
    "model__decision_function_shape": ["ovo", "ovr"],
    "model__coef0": [0, 1, 2, 3, 4, 5,10],
    "model__degree": [1, 2, 3, 4, 5,10,20],
    "model__kernel": ['poly', "rbf", "linear"],
    "model__C": loguniform(1e-2, 1e2),
    "model__gamma": ["scale", "auto"] + list(loguniform(1e-4, 1e1).rvs(10)),
    "model__shrinking": [False,True],
    "model__tol": loguniform(1e-5, 1e-1),
}



In [ ]:
NOTOPTPARAMATERS=True

if NOTOPTPARAMATERS:
    bestparameters={'model__C': np.float64(8.54422003464809), 'model__coef0': 5, 'model__decision_function_shape': 'ovo', 'model__degree': 5, 'model__gamma': np.float64(0.00033739898121480747), 'model__kernel': 'rbf', 'model__shrinking': False, 'model__tol': np.float64(0.010258725065771445)}
    svmlbp_lpq=SVC(C=bestparameters['model__C'],coef0=bestparameters['model__coef0'],degree=bestparameters['model__degree'],kernel=bestparameters["model__kernel"])
else:
    svmlbp_lpq=randomgridsearchotimizator(parameters=parameters,pipe=pipe,scoring="f1_macro",cvparts=CVPARTS,n_iter=25)

In [ ]:
svmlbp_lpq.fit(X=X_train_model,y=y_train)

In [42]:
ypredsvmglcm=svmlbp_lpq.predict(X=X_test_model)

In [43]:
print("\tSVM (LBP+LPQ+GLCM) \n")
print(classification_report(y_test,y_pred=ypredsvmglcm)+"\n")

	SVM (LBP+LPQ+GLCM) 

              precision    recall  f1-score   support

           a       0.60      0.57      0.58        44
          ad       0.61      0.53      0.57        57
           d       0.49      0.60      0.54        48
           f       0.23      0.22      0.22        41
           j       0.26      0.29      0.28        48
          jd       0.40      0.45      0.42        53
          jt       0.52      0.49      0.51        51
           m       0.43      0.37      0.40        57
          md       0.33      0.20      0.25        50
           n       0.65      0.58      0.61        52
           o       0.49      0.60      0.54        52
           s       0.36      0.47      0.41        47

    accuracy                           0.45       600
   macro avg       0.45      0.45      0.44       600
weighted avg       0.45      0.45      0.45       600




In [ ]:
pipe=Pipeline(steps=[("scalar",StandardScaler()),
                     ("knn",KNeighborsClassifier())]) 
parameters = {
    "knn__n_neighbors": [3,5,7,10,15,20,25,30],  
    "knn__weights": ['uniform', 'distance'],
    "knn__metric": ['euclidean', 'manhattan'], 
    "knn__algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute'],
    "knn__leaf_size": [10, 20, 30, 50,60,70,80,90],  
    "knn__p": [1, 2]  
}



In [ ]:

NOTOPTPARAMATERS=False

if NOTOPTPARAMATERS:
   parameters={'knn__leaf_size': 30, 'knn__n_neighbors': 8, 'knn__p': 1, 'knn__weights': 'distance'}
   knn_lbp_lqb_glcm=KNeighborsClassifier(n_neighbors=parameters['knn__n_neighbors'],leaf_size=parameters['knn__leaf_size'],p=parameters['knn__p'],weights=parameters["knn__weights"])
else:
    knn_lbp_lqb_glcm=randomgridsearchotimizator(parameters=parameters,pipe=pipe,scoring="f1_macro",cvparts=CVPARTS,n_iter=55)

In [ ]:
labelencoder=LabelEncoder()

ln_y=labelencoder.fit_transform(y_train)

In [ ]:
knn_lbp_lqb_glcm.fit(X=X_train_model,y=ln_y)

RandomizedSearchCV(cv=5, error_score='raise',
                   estimator=Pipeline(steps=[('scalar', StandardScaler()),
                                             ('knn', KNeighborsClassifier())]),
                   n_iter=55, n_jobs=6,
                   param_distributions={'knn__algorithm': ['auto', 'ball_tree',
                                                           'kd_tree', 'brute'],
                                        'knn__leaf_size': [10, 20, 30, 50],
                                        'knn__metric': ['euclidean',
                                                        'manhattan'],
                                        'knn__n_neighbors': [3, 5, 7, 10, 15],
                                        'knn__p': [1, 2],
                                        'knn__weights': ['uniform',
                                                         'distance']},
                   scoring='f1_macro')

In [ ]:

ln_y_test=labelencoder.fit_transform(y_test)
print(knn_lbp_lqb_glcm.best_params_)
print(knn_lbp_lqb_glcm.best_score_)

{'knn__weights': 'uniform', 'knn__p': 1, 'knn__n_neighbors': 7, 'knn__metric': 'manhattan', 'knn__leaf_size': 50, 'knn__algorithm': 'brute'}
0.445358459636983


In [ ]:
ypredknnglcm=knn_lbp_lqb_glcm.predict(X=X_test_model)

In [ ]:
print("\tKNN(LBP+LPQ+GLCM) \n")
print(classification_report(y_true=ln_y_test,y_pred=ypredknnglcm)+"\n")

	KNN(LBP+LPQ+GLCM) 

              precision    recall  f1-score   support

           0       0.72      0.35      0.47        37
           1       0.54      0.45      0.49        31
           2       0.40      0.73      0.51        26
           3       0.53      0.36      0.43        28
           4       0.33      0.44      0.38        27
           5       0.44      0.46      0.45        35
           6       0.40      0.29      0.33        21
           7       0.90      0.56      0.69        34
           8       0.60      0.75      0.67        24
           9       0.37      0.77      0.49        30
          10       0.30      0.19      0.23        37
          11       0.40      0.33      0.36        30

    accuracy                           0.46       360
   macro avg       0.49      0.47      0.46       360
weighted avg       0.50      0.46      0.46       360




In [ ]:

with open(f"Models/svm_LBP_LPQ_GLCM.pkl",mode="wb") as fsvm:
    pickle.dump(svmlbp_lpq,fsvm)

with open("Models/knn_LBP_LPQ_GLCM.pkl",mode="wb") as fknn:
    pickle.dump(knn_lbp_lqb_glcm,fknn)